<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/google/Gemini_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini API: Function Calling

Function calling lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with. Function calling lets you use functions as tools in generative AI applications, and you can define more than one function within a single request.

https://ai.google.dev/gemini-api/docs/function-calling

## Setup

In [1]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 6.7 MB/s eta 0:00:00


In [2]:
import google.generativeai as genai

In [3]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

To use function calling, pass a list of functions to the `tools` parameter when creating a [`GenerativeModel`](https://ai.google.dev/api/python/google/generativeai/GenerativeModel). The model uses the function name, docstring, parameters, and parameter type annotations to decide if it needs the function to best answer a prompt.

> Important: The SDK converts function parameter type annotations to a format the API understands (`genai.protos.FunctionDeclaration`). The API only supports a limited selection of parameter types, and the Python SDK's automatic conversion only supports a subset of that: `AllowedTypes = int | float | bool | str | list['AllowedTypes'] | dict`

In [36]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=[add, subtract, multiply, divide]
)



model2 = genai.GenerativeModel(
    model_name="gemini-1.5-flash")

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x7f374058fbe0>,
    system_instruction=None,
    cached_content=None
)

Function calls naturally fit in to [multi-turn chats](https://ai.google.dev/api/python/google/generativeai/GenerativeModel#multi-turn) as they capture a back and forth interaction between the user and model. The Python SDK's [`ChatSession`](https://ai.google.dev/api/python/google/generativeai/ChatSession) is a great interface for chats because handles the conversation history for you, and using the parameter `enable_automatic_function_calling` simplifies function calling even further.

In [37]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [38]:
response = chat.send_message(
    """
    You are a clever calculator.

    I have two questions and answer both. explain the secuence of step you took.
    Problem statement:
    I bought 57 boxes of wine, each one has 12 bottles. If the cost per bottle is 5 dollars, but if you buy more than 30, then individual price it is reduce to 4 to all of them.

    Questions:
    1)how many bottels I bought?
    2)how much I have paid to buy the wine"""
)
print(response.text)

I paid 2766 dollars. Here's how I solved it: 
1. I calculated the total number of bottles: 57 boxes * 12 bottles/box = 684 bottles.
2. I calculated the cost of the first 30 bottles: 30 bottles * $5/bottle = $150
3. I calculated the number of bottles that were discounted: 684 bottles - 30 bottles = 654 bottles
4. I calculated the cost of the discounted bottles: 654 bottles * $4/bottle = $2616
5. I added the cost of the first 30 bottles and the discounted bottles: $150 + $2616 = $2766 



In [12]:
57 *12, 57 * 12 *4

(684, 2736)

In [39]:
chat2 = model2.start_chat()

In [40]:
response2 = chat2.send_message(
    """
    You are a clever calculator.

    I have two questions and answer both. explain the secuence of step you took.
    Problem statement:
    I bought 57 boxes of wine, each one has 12 bottles. If the cost per bottle is 5 dollars, but if you buy more than 30, then individual price it is reduce to 4 to all of them.

    Questions:
    1)how many bottels I bought?
    2)how much I have paid to buy the wine"""
)
print(response2.text)

Here's how we can solve this problem:

**1) How many bottles did you buy?**

* **Step 1:** Calculate the total number of bottles: 57 boxes * 12 bottles/box = 684 bottles

**2) How much did you pay for the wine?**

* **Step 1:** Determine how many bottles qualify for the discount: 684 bottles - 30 bottles = 654 bottles
* **Step 2:** Calculate the cost of the discounted bottles: 654 bottles * $4/bottle = $2616
* **Step 3:** Calculate the cost of the non-discounted bottles: 30 bottles * $5/bottle = $150
* **Step 4:** Add the cost of both types of bottles: $2616 + $150 = $2766

**Answer:**

1. You bought **684** bottles.
2. You paid **$2766** for the wine. 



The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`genai.protos.Content`](https://ai.google.dev/api/python/google/generativeai/protos/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`genai.protos.Part`](https://ai.google.dev/api/python/google/generativeai/protos/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`genai.protos.FunctionCall`](https://ai.google.dev/api/python/google/generativeai/protos/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`genai.protos.FunctionResponse`](https://ai.google.dev/api/python/google/generativeai/protos/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [41]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': '\n    You are a clever calculator.\n\n    I have two questions and answer both. explain the secuence of step you took.\n    Problem statement:\n    I bought 57 boxes of wine, each one has 12 bottles. If the cost per bottle is 5 dollars, but if you buy more than 30, then individual price it is reduce to 4 to all of them.\n\n    Questions:\n    1)how many bottels I bought?\n    2)how much I have paid to buy the wine'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 12.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 684.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'I bought 684 bottles. \n'}, {'function_call': {'name': 'multiply', 'args': {'b': 5.0, 'a': 30.0}}}]
-----------------------------

## Single Function


Gemini has the ability to call user-defined functions.

In [18]:
def get_full_menu(service: str):
    """List all items on the menu of Gemini's Trattoria for the given service.

    Args:
        name: The type of service, lunch or dinner.
    """
    return ["Chicken Caesar Salad", "Margherita Pizza", "Spaghetti and Meatballs", "Eggplant Parmesan"]


def find_vegetarian_items(items: list[str]):
    """List all dishes in items that are vegetarian.

    Args:
        items: A list of dinner dishes.
    """
    return ["Margherita Pizza", "Eggplant Parmesan"]

def enter_restaurant():
    """You enter Gemini's Trattoria, moving the creaky door."""
    print("The door swings open, making a loud noise.")
    return True

functions = {"get_full_menu": get_full_menu,
             "find_vegetarian_items": find_vegetarian_items,
             "enter_restaurant": enter_restaurant}

After this, we go ahead and define our Gemini model. Notice how we include the argument for tools, which tells the model which functions it has available to use. We create a chat and set automatic function calling to True, which we will touch on later.

In [19]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=functions.values()
)

chat = model.start_chat(enable_automatic_function_calling=True)

Now, we go ahead and send a prompt that requires the model to call our user-defined functions.

In [20]:
response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)
print(response.text)

The dinner menu includes: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggplant Parmesan. 



As we can see in the chat history, the model initially sends back a function call, to which we automatically respond, which then leads to the final model output.

In [21]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items are on Gemini's Trattoria's dinner menu?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'The dinner menu includes: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, and Eggp

## Behind The Scenes

In general, the processes goes as follows:
1. The user submits a query to the model.
2. The model responds with a function call.
3. The user runs the function and returns the result of the function.
4. Now, the model will either go back to Step 2 or output a final response, as seen above.

<div>
<img src="https://video.udacity-data.com/topher/2024/June/66749652_function_calling/function_calling.jpeg" width="500"/>
</div>

We can see this step-by-step in action by running the previous example with manual function calling, by setting the automatic function calling argument to False.

In [22]:
chat = model.start_chat(enable_automatic_function_calling=False)

response = chat.send_message(
    "What items are on Gemini's Trattoria's dinner menu?"
)

part = response.candidates[0].content.parts[0]
part

function_call {
  name: "get_full_menu"
  args {
    fields {
      key: "service"
      value {
        string_value: "dinner"
      }
    }
  }
}

After this, we can reply to the model as specified in Step 3, using the functions dictionary we made earlier.

In [23]:
import google.ai.generativelanguage as glm
from google.protobuf.struct_pb2 import Struct

# Put the result in a protobuf Struct
s = Struct()
result = functions[part.function_call.name](**part.function_call.args)
s.update({"result": result})

function_response = glm.Part(
    function_response=glm.FunctionResponse(name="get_full_menu", response=s)
)

# Generate the next response
response = chat.send_message(function_response)
print(response.text)

The dinner menu includes: Chicken Caesar Salad, Margherita Pizza, Spaghetti and Meatballs, Eggplant Parmesan. 



## Multiple Function Calls

Our model can also call multiple functions in a row, either at the same time or one after the other, as we see in both cases below.

In [24]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are vegetarian?"
)
print(response.text)

The vegetarian items on the dinner menu are Margherita Pizza and Eggplant Parmesan. 



In [26]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}, {'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'enter_restaurant', 'response': {'result': True}}}, {'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------

In [27]:
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."
)
print(response.text)

The door swings open, making a loud noise.
I enter the restaurant, pushing open the creaky door. The aroma of garlic and herbs hits me immediately. It's a cozy little place, full of chatter and laughter. Looking at the menu, I see the following items:

* Chicken Caesar Salad
* Margherita Pizza
* Spaghetti and Meatballs
* Eggplant Parmesan 



In [28]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Your are standing outside of Gemini's Trattoria. Enter the restaurant and read out the items on the menu."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}, {'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'enter_restaurant', 'response': {'result': True}}}, {'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------

## Function Calling Config

# Function calling mode
You can use the Function Calling mode parameter to modify the execution behavior of the feature. There are three modes available:

- AUTO: The default model behavior. The model decides to predict either a function call or a natural language response.
- ANY: The model is constrained to always predict a function call. If allowed_function_names is not provided, the model picks from all of the available function declarations. If allowed_function_names is provided, the model picks from the set of allowed functions.
- NONE: The model won't predict a function call. In this case, the model behavior is the same as if you don't pass any function declarations.

You can also pass a set of allowed_function_names that, when provided, limits the functions that the model will call. You should only include allowed_function_names when the mode is ANY. Function names should match function declaration names. With the mode set to ANY and the allowed_function_names set, the model will predict a function call from the set of function names provided

In [31]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

First, we use the NONE mode, which tells the model to not make any function calls. In this example, the model knows about the get_full_menu function but is unable to use it.

In [32]:
tool_config = tool_config_from_mode("none")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are vegetarian?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Gemini's Trattoria's dinner menu are vegetarian?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': "Please provide me with the dinner menu of Gemini's Trattoria so I can identify the vegetarian options. \n"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


AUTO mode lets the model decide whether to reply with text or to call specific functions. In this example, we see that the model calls a function to get all menu items but is able to reason on its own on which items are pasta dishes.

In [33]:
tool_config = tool_config_from_mode("auto")
chat = model.start_chat(enable_automatic_function_calling=True)

response = chat.send_message(
    "What items on Gemini's Trattoria's dinner menu are pasta dishes?", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "What items on Gemini's Trattoria's dinner menu are pasta dishes?"}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'dinner'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'get_full_menu', 'response': {'result': ['Chicken Caesar Salad', 'Margherita Pizza', 'Spaghetti and Meatballs', 'Eggplant Parmesan']}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'text': 'The dinner menu has one pasta dish: Spaghetti and Meatballs. \n'}]
-------------------

Finally, ANY mode forces the model to make function calls, as seen in the below example. We decide to switch over to the Gemini 1.5 Pro model to achieve the intended behavior.

In [34]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-pro", tools=functions.values()
)

tool_config = tool_config_from_mode("any")
chat = model.start_chat()

response = chat.send_message(
    "Enter Gemini's Trattoria.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Enter Gemini's Trattoria."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'enter_restaurant', 'args': {}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


By setting allowed_function_names, the model will only choose from those functions. If it is not set, all of the functions in tools are candidates for function calling. In the below example, since we are in ANY mode, the model is forced to use a function, even though it is not necessarily the best option.

In [35]:
tool_config = tool_config_from_mode("any", ["get_full_menu"])
chat = model.start_chat()

response = chat.send_message(
    "Enter Gemini's Trattoria.", tool_config=tool_config
)

for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 180)

user -> [{'text': "Enter Gemini's Trattoria."}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'get_full_menu', 'args': {'service': 'lunch'}}}]
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
